# Scenario Generation for Decision Optimization Experiment

To use this Notebook you must have already created a DO experiment and one scenario from 4-DO_Model_Builder


### Client

Create a DODS client to connect to initial scenario.

In [1]:
from dd_scenario import *

client = Client()
decision = client.get_model_builder(name="Supply Chain Optimization")
scenario = decision.get_scenario(name="Scenario - Predicted Demand")

### High Demand Scenario

In [2]:
import pandas as pd
import random as random

high_demand = pd.read_csv('/project_data/data_asset/DO_Route_Data.csv')
high_demand.head()

Warehouse  Product_Code SupplierID ManufID WH1 WH2 WH3 WH4  Time_Enroute  \
0        W2  Product_0001         S1      M1  W1  W3  W4  W2           5.0   
1        W2  Product_0001         S1      M1  W1  W3  W4  W2           5.0   
2        W2  Product_0001         S1      M1  W1  W4  W3  W2           5.0   
3        W2  Product_0001         S1      M1  W1  W4  W3  W2           5.0   
4        W2  Product_0001         S1      M1  W3  W1  W4  W2           5.0   

  Destination        Miles           f1           f2           f3  \
0          W2  2041.694319  2041.694319  2001.785173  1941.680672   
1          W2  2041.694319  2041.694319  2001.785173  1941.680672   
2          W2  1338.990849  1338.990849  1299.081702  1238.977201   
3          W2  1338.990849  1338.990849  1299.081702  1238.977201   
4          W2  2050.240376  2050.240376  2010.331229  1584.740888   

            f4          f5  capacity                     prodx  cost  Forecast  
0  1574.236047  965.635097    1403.0  Product_0001S1M1W1W3W4W2  0.02    1750.0  
1  1574.236047  965.635097     187.0  Product_0001S1M1W1W3W4W2  0.02    1750.0  
2   987.316035  378.715085    1403.0  Product_0001S1M1W1W4W3W2  0.02    1750.0  
3   987.316035  378.715085     187.0  Product_0001S1M1W1W4W3W2  0.02    1750.0  
4  1217.296263  965.635097    1403.0  Product_0001S1M1W3W1W4W2  0.02    1750.0

First, lets increase demand.
In lieu of a confidence interval, we will use a randomly generated percentage increase in demand across all products

In [3]:
value = [random.randrange(10, 100, 1)/1000 for i in range(len(high_demand))]
value = [x + 1 for x in value]
high_demand['Forecast'] = round(high_demand['Forecast'] * value)
high_demand.head()

Warehouse  Product_Code SupplierID ManufID WH1 WH2 WH3 WH4  Time_Enroute  \
0        W2  Product_0001         S1      M1  W1  W3  W4  W2           5.0   
1        W2  Product_0001         S1      M1  W1  W3  W4  W2           5.0   
2        W2  Product_0001         S1      M1  W1  W4  W3  W2           5.0   
3        W2  Product_0001         S1      M1  W1  W4  W3  W2           5.0   
4        W2  Product_0001         S1      M1  W3  W1  W4  W2           5.0   

  Destination        Miles           f1           f2           f3  \
0          W2  2041.694319  2041.694319  2001.785173  1941.680672   
1          W2  2041.694319  2041.694319  2001.785173  1941.680672   
2          W2  1338.990849  1338.990849  1299.081702  1238.977201   
3          W2  1338.990849  1338.990849  1299.081702  1238.977201   
4          W2  2050.240376  2050.240376  2010.331229  1584.740888   

            f4          f5  capacity                     prodx  cost  Forecast  
0  1574.236047  965.635097    1403.0  Product_0001S1M1W1W3W4W2  0.02    1810.0  
1  1574.236047  965.635097     187.0  Product_0001S1M1W1W3W4W2  0.02    1890.0  
2   987.316035  378.715085    1403.0  Product_0001S1M1W1W4W3W2  0.02    1825.0  
3   987.316035  378.715085     187.0  Product_0001S1M1W1W4W3W2  0.02    1869.0  
4  1217.296263  965.635097    1403.0  Product_0001S1M1W3W1W4W2  0.02    1806.0

In order to avoid a compelte overload of the system, we will slightly increase supplier capacities by a flat 5%

In [4]:
high_demand['capacity'] = round(high_demand['capacity'] * 1.05)

Lastly, run the new scenario

In [5]:
all_kpis = pd.DataFrame()

In [6]:
sc_name = "Scenario - High Demand"
print(sc_name)
copy = decision.get_scenario(name=sc_name)
if (copy != None):
    print(" Deleting old...")
    decision.delete_container(copy)
print(" Copying from original scenario...")    
copy = scenario.copy(sc_name)
print(" Generating new demand...")
copy.add_table_data("DO_Route_Data", high_demand, category='input')
print(" Solving...")
copy.solve()
print(" Grabbing solution kpis...")
kpis = copy.get_table_data('kpis')
kpis['scenario'] = sc_name
mk = [[ kpis.iloc[0]['Value'],  "Scenario - High Demand" % (kpis.iloc[1]['Value']), sc_name, "Scenario - High Demand" % (kpis.iloc[2]['Value'])]]
my_kpis = pd.DataFrame(data=mk, columns=['cost','fix','scenario','temp'])
copy.add_table_data('my_kpis', data=my_kpis, category='output')
all_kpis = all_kpis.append(kpis)
print("Done!")

Scenario - High Demand
 Deleting old...
 Copying from original scenario...
 Generating new demand...
 Solving...
 Grabbing solution kpis...
Done!


### Low Demand Scenario

In [7]:
low_demand = pd.read_csv('/project_data/data_asset/DO_Route_Data.csv')
low_demand.head()

Warehouse  Product_Code SupplierID ManufID WH1 WH2 WH3 WH4  Time_Enroute  \
0        W2  Product_0001         S1      M1  W1  W3  W4  W2           5.0   
1        W2  Product_0001         S1      M1  W1  W3  W4  W2           5.0   
2        W2  Product_0001         S1      M1  W1  W4  W3  W2           5.0   
3        W2  Product_0001         S1      M1  W1  W4  W3  W2           5.0   
4        W2  Product_0001         S1      M1  W3  W1  W4  W2           5.0   

  Destination        Miles           f1           f2           f3  \
0          W2  2041.694319  2041.694319  2001.785173  1941.680672   
1          W2  2041.694319  2041.694319  2001.785173  1941.680672   
2          W2  1338.990849  1338.990849  1299.081702  1238.977201   
3          W2  1338.990849  1338.990849  1299.081702  1238.977201   
4          W2  2050.240376  2050.240376  2010.331229  1584.740888   

            f4          f5  capacity                     prodx  cost  Forecast  
0  1574.236047  965.635097    1403.0  Product_0001S1M1W1W3W4W2  0.02    1750.0  
1  1574.236047  965.635097     187.0  Product_0001S1M1W1W3W4W2  0.02    1750.0  
2   987.316035  378.715085    1403.0  Product_0001S1M1W1W4W3W2  0.02    1750.0  
3   987.316035  378.715085     187.0  Product_0001S1M1W1W4W3W2  0.02    1750.0  
4  1217.296263  965.635097    1403.0  Product_0001S1M1W3W1W4W2  0.02    1750.0

First, lets decrease demand.
In lieu of a confidence interval, we will use a randomly generated percentage decrease in demand across all products

In [8]:
value = [random.randrange(10, 100, 1)/1000 for i in range(len(low_demand))]
value = [1 - x for x in value]
low_demand['Forecast'] = round(low_demand['Forecast'] * value)
low_demand.head()

Warehouse  Product_Code SupplierID ManufID WH1 WH2 WH3 WH4  Time_Enroute  \
0        W2  Product_0001         S1      M1  W1  W3  W4  W2           5.0   
1        W2  Product_0001         S1      M1  W1  W3  W4  W2           5.0   
2        W2  Product_0001         S1      M1  W1  W4  W3  W2           5.0   
3        W2  Product_0001         S1      M1  W1  W4  W3  W2           5.0   
4        W2  Product_0001         S1      M1  W3  W1  W4  W2           5.0   

  Destination        Miles           f1           f2           f3  \
0          W2  2041.694319  2041.694319  2001.785173  1941.680672   
1          W2  2041.694319  2041.694319  2001.785173  1941.680672   
2          W2  1338.990849  1338.990849  1299.081702  1238.977201   
3          W2  1338.990849  1338.990849  1299.081702  1238.977201   
4          W2  2050.240376  2050.240376  2010.331229  1584.740888   

            f4          f5  capacity                     prodx  cost  Forecast  
0  1574.236047  965.635097    1403.0  Product_0001S1M1W1W3W4W2  0.02    1670.0  
1  1574.236047  965.635097     187.0  Product_0001S1M1W1W3W4W2  0.02    1690.0  
2   987.316035  378.715085    1403.0  Product_0001S1M1W1W4W3W2  0.02    1656.0  
3   987.316035  378.715085     187.0  Product_0001S1M1W1W4W3W2  0.02    1722.0  
4  1217.296263  965.635097    1403.0  Product_0001S1M1W3W1W4W2  0.02    1657.0

Run the new scenario

In [9]:
all_kpis = pd.DataFrame()

In [10]:
sc_name = "Scenario - Low Demand"
print(sc_name)
copy = decision.get_scenario(name=sc_name)
if (copy != None):
    print(" Deleting old...")
    decision.delete_container(copy)
print(" Copying from original scenario...")    
copy = scenario.copy(sc_name)
print(" Generating new demand...")
copy.add_table_data("DO_Route_Data", low_demand, category='input')
print(" Solving...")
copy.solve()
print(" Grabbing solution kpis...")
kpis = copy.get_table_data('kpis')
kpis['scenario'] = sc_name
mk = [[ kpis.iloc[0]['Value'],  "Scenario - Low Demand" % (kpis.iloc[1]['Value']), sc_name, "Scenario - Low Demand" % (kpis.iloc[2]['Value'])]]
my_kpis = pd.DataFrame(data=mk, columns=['cost','fix','scenario','temp'])
copy.add_table_data('my_kpis', data=my_kpis, category='output')
all_kpis = all_kpis.append(kpis)
print("Done!")

Scenario - Low Demand
 Deleting old...
 Copying from original scenario...
 Generating new demand...
 Solving...
 Grabbing solution kpis...
Done!


**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020 <br>
All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**